# Continuous training with TFX and Google Cloud AI Platform

## Learning Objectives

1.  Use the TFX CLI to build a TFX pipeline.
2.  Deploy a TFX pipeline version without tuning to a hosted AI Platform Pipelines instance.
3.  Create and monitor a TFX pipeline run using the TFX CLI.
4.  Deploy a new TFX pipeline version with tuning enabled to a hosted AI Platform Pipelines instance.
5.  Create and monitor another TFX pipeline run directly in the KFP UI.

In this lab, you use utilize the following tools and services to deploy and run a TFX pipeline on Google Cloud that automates the development and deployment of a TensorFlow 2.3 WideDeep Classifer to predict forest cover from cartographic data:

* The [**TFX CLI**](https://www.tensorflow.org/tfx/guide/cli) utility to build and deploy a TFX pipeline.
* A hosted [**AI Platform Pipeline instance (Kubeflow Pipelines)**](https://www.tensorflow.org/tfx/guide/kubeflow) for TFX pipeline orchestration.
* [**Dataflow**](https://cloud.google.com/dataflow) jobs for scalable, distributed data processing for TFX components.
* A [**AI Platform Training**](https://cloud.google.com/ai-platform/) job for model training and flock management for parallel tuning trials. 
* [**AI Platform Prediction**](https://cloud.google.com/ai-platform/) as a model server destination for blessed pipeline model versions.
* [**CloudTuner**](https://www.tensorflow.org/tfx/guide/tuner#tuning_on_google_cloud_platform_gcp) and [**AI Platform Vizier**](https://cloud.google.com/ai-platform/optimizer/docs/overview) for advanced model hyperparameter tuning using the Vizier algorithm.

You will then create and monitor pipeline runs using the TFX CLI as well as the KFP UI.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
print(sys.path)

['/home/michal/PycharmProjects/ml-gcp-pipeline/tfx_titanic_pipeline', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/michal/venv/ML-3.8/lib/python3.8/site-packages', '/usr/local/lib/python3.8/dist-packages', '/usr/lib/python3/dist-packages', '/home/michal/venv/ML-3.8/lib/python3.8/site-packages/IPython/extensions', '/home/michal/.ipython']


### Setup

#### Update lab environment PATH to include TFX CLI and skaffold

In [3]:
import yaml

# Set `PATH` to include the directory containing TFX CLI and skaffold.
PATH=%env PATH
HOME=%env HOME
#PYTHONPATH=%env PYTHONPATH 

%env PATH={HOME}/.local/bin:{PATH}
#%env PYTHONPATH=/home/michal/PycharmProjects/ml-gcp-pipeline:{PYTHONPATH}

env: PATH=/home/michal/.local/bin:/home/michal/venv/ML-3.8/bin:/home/michal/google-cloud-sdk/bin:/home/michal/anaconda3/bin:/home/michal/anaconda3/condabin:/home/michal/.local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin


#### Validate lab package version installation

In [4]:
!python -c "import tfx; print('TFX version: {}'.format(tfx.__version__))"
!python -c "import kfp; print('KFP version: {}'.format(kfp.__version__))"

TFX version: 0.25.0
KFP version: 1.0.4


**Note**: this lab was built and tested with the following package versions:

`TFX version: 0.25.0`  
`KFP version: 1.0.4`

## Setup local path to data, train, test folders 

In [5]:
import os
from pathlib import Path

notebook_path=os.getcwd()
local_data_dirpath = os.path.join(notebook_path, 'data')

local_train_dirpath = os.path.join(local_data_dirpath, "train")
local_train_filepath = os.path.join(local_train_dirpath, "train.csv")
local_test_dirpath = os.path.join(local_data_dirpath, "test")
local_test_filepath = os.path.join(local_test_dirpath, "test.csv")


## Load kaggle

In [ ]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!rm kaggle.json
!chmod 600 ~/.kaggle/kaggle.json


## Download data from kaggle, unzip it and copy it to data folder

In [ ]:
!kaggle competitions download -c titanic -p {local_data_dirpath} --force
!unzip -o {local_data_dirpath}/"titanic.zip" -d {local_data_dirpath}
!cp {local_data_dirpath}/"train.csv" {local_train_filepath}
!cp {local_data_dirpath}/"test.csv" {local_test_filepath}

# clean up
!rm  {local_data_dirpath}/*.csv  {local_data_dirpath}/*.zip

## Copy data to gs

In [8]:
PIPELINE_NAME = 'tfx-titanic-training'
MODEL_NAME = 'tfx_titanic_classifier'
DATA_ROOT_URI = local_train_dirpath
RUNTIME_VERSION = '2.3'
PYTHON_VERSION = '3.7'
ENABLE_CACHE=True

TRAIN_STEPS = '10000'
EVAL_STEPS = '1000'
TRAIN_BATCH_SIZE = '64'
EVAL_BATCH_SIZE = '64'

ENABLE_TUNING=False
TUNER_STEPS = '2000'
MAX_TRIALS = '50'


In [9]:
%env PIPELINE_NAME={PIPELINE_NAME}
%env MODEL_NAME={MODEL_NAME}
%env DATA_ROOT_URI={DATA_ROOT_URI}
%env RUNTIME_VERSION={RUNTIME_VERSION}
%env PYTHON_VERSION={PYTHON_VERSION}
%env ENABLE_CACHE={ENABLE_CACHE}

%env TRAIN_STEPS={TRAIN_STEPS}
%env EVAL_STEPS={EVAL_STEPS}
%env TRAIN_BATCH_SIZE={TRAIN_BATCH_SIZE}
%env EVAL_BATCH_SIZE={EVAL_BATCH_SIZE}


%env ENABLE_TUNING={ENABLE_TUNING}
%env TUNER_STEPS={TUNER_STEPS}
%env MAX_TRIALS={MAX_TRIALS}




env: PIPELINE_NAME=tfx-titanic-training
env: MODEL_NAME=tfx_titanic_classifier
env: DATA_ROOT_URI=/home/michal/PycharmProjects/ml-gcp-pipeline/tfx_titanic_pipeline/data/train
env: RUNTIME_VERSION=2.3
env: PYTHON_VERSION=3.7
env: ENABLE_CACHE=True
env: TRAIN_STEPS=10000
env: EVAL_STEPS=1000
env: TRAIN_BATCH_SIZE=64
env: EVAL_BATCH_SIZE=64
env: ENABLE_TUNING=False
env: TUNER_STEPS=2000
env: MAX_TRIALS=50


## Local pipeline run

In [10]:
%cd {notebook_path}/pipeline

/home/michal/PycharmProjects/ml-gcp-pipeline/tfx_titanic_pipeline/pipeline


In [10]:
#import pipeline.local_runner

!python local_runner.py

2021-04-16 13:37:53.583053: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
INFO:absl:PIPELINE_ROOT=/home/michal/artifact-store/tfx-titanic-training/20210416_133756
INFO:absl:Cleaning local log folder : /tmp/logs
INFO:absl:train_steps for training: 10000
INFO:absl:tuner_steps for tuning: 2000
INFO:absl:data_root_uri for training: /home/michal/PycharmProjects/ml-gcp-pipeline/tfx_titanic_pipeline/data/train
INFO:absl:eval_steps for evaluating: 1000
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:tuner_args: {'module_file': 'model.py', 'examples': Channel(
    type_name: Examples
    artifacts: []
), 'transform_graph': Channel(
    type_name: TransformGraph
    artifacts: []
), 'train_args': {'num_steps': 2000}, 'eval_args': {'num_steps': 1000}, 'custom_config': {'m

INFO:absl:TFT preprocessing. Mean value for Age = Tensor("mean/mean_and_var/Placeholder:0", shape=(), dtype=float32)
INFO:absl:TFT preprocessing. Mean value for Fare = Tensor("mean_1/mean_and_var/Placeholder:0", shape=(), dtype=float32)
INFO:absl:TFT preprocessing. Mean value for Age = Tensor("mean/mean_and_var/Placeholder:0", shape=(), dtype=float32)
INFO:absl:TFT preprocessing. Mean value for Fare = Tensor("mean_1/mean_and_var/Placeholder:0", shape=(), dtype=float32)
INFO:absl:Feature Embarked has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Ticket has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Sex has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Name has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Cabin has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Age has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Fare has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Parch has no shape. Setting

Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'Counter' object has no attribute 'name'
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'Counter' object has no attribute 'name'
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'Counter' object has no attribute 'name'
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'Counter' object has no attribute 'name'
INFO:absl:Feature Embarked has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Ticket has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Sex has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Name has no shape. Setting to VarLenSparseTensor.

INFO:absl:Running publisher for Transform
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Component Transform is finished.
INFO:absl:Component ExampleValidator is running.
INFO:absl:Running driver for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for ExampleValidator
INFO:absl:Validating schema against the computed statistics for split train.
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
INFO:absl:Validation complete for split train. Anomalies written to /home/michal/artifact-store/tfx-titanic-training/20210416_133756/ExampleValidator/anomalies/6/train.
INFO:absl:Validating schema against the computed statistics for split eval.
INFO:absl:Validation complete for split eval. Anomalies written to /home/michal/artifact-store/tfx-titanic-training/20210416_133756/ExampleValidator/anomalies/6/eval.
INF

INFO:absl:Feature Age_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Embarked_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Fare_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Parch_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Pclass_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Sex_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature SibSp_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Survived_xf has a shape . Setting to DenseTensor.
Search space summary
Default search space size: 5
learning_rate (Float)
{'default': 0.0009167702421017742, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}
n_layers (Int)
{'default': 2, 'conditions': [], 'min_value': 1, 'max_value': 2, 'step': 1, 'sampling': None}
n_units_1 (Int)
{'default': 72, 'conditions': [{'class_name': 'Parent', 'config': {'name': 'n_layers', 'values': [1]}}], 'min_value': 8, 'max_value': 128, 'step': 8, 'samp

...

2000/2000 [==============================] - 82s 41ms/step - loss: 0.4552 - tp: 31432.0000 - fp: 9238.0000 - tn: 71543.0000 - fn: 15787.0000 - binary_accuracy: 0.8045 - precision: 0.7729 - recall: 0.6657 - auc: 0.8402 - val_loss: 0.4064 - val_tp: 21460.0000 - val_fp: 6023.0000 - val_tn: 29363.0000 - val_fn: 7154.0000 - val_binary_accuracy: 0.7941 - val_precision: 0.7808 - val_recall: 0.7500 - val_auc: 0.8925

Trial 48 Complete [00h 01m 25s]
val_binary_accuracy: 0.7941094040870667

Best val_binary_accuracy So Far: 0.8059687614440918
Total elapsed time: 01h 08m 54s

Search: Running Trial #49

Hyperparameter    |Value             |Best Value So Far 
learning_rate     |0.0030039         |0.00041834        
n_layers          |2                 |2                 
n_units_1         |48                |56                
n_units_2         |32                |88                

INFO:absl:Model: "functional_1"
INFO:absl:__________________________________________________________________________

INFO:absl:==================================================================================================
INFO:absl:Total params: 3,403
INFO:absl:Trainable params: 3,403
INFO:absl:Non-trainable params: 0
INFO:absl:__________________________________________________________________________________________________
2000/2000 [==============================] - 84s 42ms/step - loss: 0.4583 - tp: 31127.0000 - fp: 8980.0000 - tn: 71789.0000 - fn: 16104.0000 - binary_accuracy: 0.8040 - precision: 0.7761 - recall: 0.6590 - auc: 0.8385 - val_loss: 0.4063 - val_tp: 21459.0000 - val_fp: 6026.0000 - val_tn: 29361.0000 - val_fn: 7154.0000 - val_binary_accuracy: 0.7941 - val_precision: 0.7808 - val_recall: 0.7500 - val_auc: 0.8922

Trial 50 Complete [00h 01m 27s]
val_binary_accuracy: 0.7940624952316284

Best val_binary_accuracy So Far: 0.8059687614440918
Total elapsed time: 01h 11m 56s
INFO:absl:Finished tuning... Tuner ID: tuner0
Results summary
Results in /home/michal/artifact-store/tfx-titanic-t

INFO:absl:                                                                 SibSp_xf[0][0]                   
INFO:absl:__________________________________________________________________________________________________
INFO:absl:dense_2 (Dense)                 (None, 56)           168         dense_features_2[0][0]           
INFO:absl:__________________________________________________________________________________________________
INFO:absl:dense_3 (Dense)                 (None, 88)           5016        dense_2[0][0]                    
INFO:absl:__________________________________________________________________________________________________
INFO:absl:dense_features_3 (DenseFeatures (None, 3050)         0           Age_xf[0][0]                     
INFO:absl:                                                                 Embarked_xf[0][0]                
INFO:absl:                                                                 Fare_xf[0][0]                    
INFO:absl:         

1000/1000 - 47s - loss: 0.4484 - tp: 16068.0000 - fp: 4799.0000 - tn: 35591.0000 - fn: 7542.0000 - binary_accuracy: 0.8072 - precision: 0.7700 - recall: 0.6806 - auc: 0.8439 - val_loss: 0.4065 - val_tp: 21457.0000 - val_fp: 6023.0000 - val_tn: 29370.0000 - val_fn: 7150.0000 - val_binary_accuracy: 0.7942 - val_precision: 0.7808 - val_recall: 0.7501 - val_auc: 0.8926
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
2021-04-16 14:58:34.120360: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied 

INFO:absl:Component Pusher is finished.


Run docker container for serving 

In [36]:
!docker run --rm -p 8500:8500 -p 8501:8501 -p 8503:8503 -v=1 \
 --mount type=bind,source=/home/michal/artifact-store/tfx-titanic-training/20210416_133756/Pusher/pushed_model/,target=/models/tfx_titanic_classifier \
 -e MODEL_NAME=tfx_titanic_classifier -t tensorflow/serving:latest
                

                

2021-04-10 20:30:48.816717: I tensorflow_serving/model_servers/server.cc:88] Building single TensorFlow model file config:  model_name: tfx_titanic_classifier model_base_path: /models/tfx_titanic_classifier
2021-04-10 20:30:48.816899: I tensorflow_serving/model_servers/server_core.cc:464] Adding/updating models.
2021-04-10 20:30:48.816950: I tensorflow_serving/model_servers/server_core.cc:587]  (Re-)adding model: tfx_titanic_classifier
2021-04-10 20:30:48.917580: I tensorflow_serving/core/basic_manager.cc:740] Successfully reserved resources to load servable {name: tfx_titanic_classifier version: 11}
2021-04-10 20:30:48.917636: I tensorflow_serving/core/loader_harness.cc:66] Approving load for servable version {name: tfx_titanic_classifier version: 11}
2021-04-10 20:30:48.917653: I tensorflow_serving/core/loader_harness.cc:74] Loading servable version {name: tfx_titanic_classifier version: 11}
2021-04-10 20:30:48.917703: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:32]

Functions for serializing data to tf.train.Example

In [12]:
import importlib
import client.client_util as client_util


importlib.reload(client_util)


df = client_util.read_csv_for_prediction(local_test_filepath)
df.head()

df_first_10 = df.head(10)
df_first_10_dict = df_first_10.to_dict(orient='records')


Use Prod client

In [15]:
import client.prod_client as prod_client
import client.client_util as client_util

import importlib
importlib.reload(prod_client)
importlib.reload(client_util)

prodclient = prod_client.ProdClient(host='localhost:8500', model_name='tfx_titanic_classifier', \
                                    model_version=12, \
                                    in_tensor_name='examples', signature_name='serving_default')


prodclient.predict(request_data=df_first_10_dict, request_timeout=30)



{'output_0': array([0.13674265, 0.44095069, 0.13071445, 0.0998413 , 0.67094266,
        0.14791387, 0.71151745, 0.24962968, 0.7586664 , 0.03947487])}

Use InMemory Client

In [14]:
import client.inmemory_client as inmemory_client
import client.client_util as client_util

import importlib
importlib.reload(inmemory_client)
importlib.reload(client_util)

inmemory_client = inmemory_client.InMemoryClient(
            'tests/resources/serving_model/1618577937', in_tensor_name='examples', signature_name='serving_default')



prediction = inmemory_client.predict(request_data=df_first_10_dict)

prediction



{'output_0': <tf.Tensor: shape=(10,), dtype=float32, numpy=
 array([0.13674265, 0.4409507 , 0.13071445, 0.0998413 , 0.67094266,
        0.14791387, 0.71151745, 0.24962968, 0.7586664 , 0.03947487],
       dtype=float32)>}